In [65]:
%load_ext autoreload
%autoreload 2

In [66]:
from emulation import Emulation
import numpy as np
import pandas as pd
from copy import deepcopy

pd.set_option('display.max_rows', 501)

In [67]:
import yaml
import pprint
import pyperclip as clip

def printLogs(logs):
    logs_str = yaml.dump(logs, default_flow_style=False)
    clip.copy(f"{logs}")
    print(logs_str)
    print('=========================')

def copyLogs(logs):
    clip.copy(f"{logs}")

In [69]:
def doExp(exp, Exp):
    exp = deepcopy(exp)
    logStart = exp.logState()

    dfExp, logExp = Exp(deepcopy(exp), 'exp')
    dfOk, logOk = Exp(deepcopy(exp), 'ok')

    dfRes = dfExp.copy()
    dfRes['balanceOk'] = dfOk['balanceOk']

    return dfRes, logStart, logExp, logOk

In [71]:
exp = Emulation()

### Pump and rebase

In [89]:
def Exp0(exp, mode):
    assert mode in ('exp', 'ok')

    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    pid = None
    for i in range(len(exp.router.pools)):
        if exp.tokens[1] in exp.router.pools[i]['amounts']:
            pid = i
            break
    pool = exp.router.pools[pid]
    lp = list(pool['users'].items())[0][1]
    allLps = sum([i for _, i in list(pool['users'].items())])
    depositDict = {tok: amt * lp / allLps for tok, amt in pool['amounts'].items()}
    depositTokens, depositAmounts = list(depositDict.keys()), list(depositDict.values())

    exp.host.mintTokens(depositTokens, depositAmounts)

    exp.host.deposit(0, depositTokens, depositAmounts)

    exp.farms.block += 60
    
    exp.router.changeTokenPrice(exp.tokens[1], prevBprice*100)

    if mode == 'exp': exp.host.withdraw(0)

    exp.router.changeTokenPrice(exp.tokens[1], prevBprice)
    
    if mode == 'exp': exp.host.deposit(0, depositTokens, [tok.balanceOf(exp.host) for tok in depositTokens])

    exp.farms.block += 36000

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)

    users = [exp.host.name]
    balances = [exp.doku.balanceOf(exp.host) // 1e18]

    for user in exp.users:
        if exp.doku.balanceOf(user) != 0:
            users.append(user.name)
            balances.append(exp.doku.balanceOf(user) // 1e18)

    balName = 'balanceExp' if mode == 'exp' else 'balanceOk'
    df = pd.DataFrame({'user': users, balName: balances})
    df = df.set_index('user')
    df[balName] = df[balName].astype(int)

    return df, log_end    

In [90]:
dfRes, logStart, logExp, logOk = doExp(exp, Exp0)
dfRes

,balanceExp,balanceOk
user,,
HOST,22283,15025
alice,10462,15025
bob,16230,14364
carol,19813,19813
david,30275,34838


### Price rise twice during 100 steps, HOST rebase every time

In [41]:
def Exp1(exp, mode):
    assert mode in ('exp', 'ok')

    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    pid = None
    for i in range(len(exp.router.pools)):
        if exp.tokens[1] in exp.router.pools[i]['amounts']:
            pid = i
            break
    pool = exp.router.pools[pid]
    lp = list(pool['users'].items())[0][1]
    allLps = sum([i for _, i in list(pool['users'].items())])
    depositDict = {tok: amt * lp / allLps for tok, amt in pool['amounts'].items()}
    depositTokens, depositAmounts = list(depositDict.keys()), list(depositDict.values())

    exp.host.mintTokens(depositTokens, depositAmounts)

    exp.host.deposit(0, depositTokens, depositAmounts)

    exp.farms.block += 60
    
    nIter = 100
    for i in range(nIter):
        log_prev = exp.logState()
        prevBprice = exp.router.prices[exp.tokens[1]]
        exp.router.changeTokenPrice(exp.tokens[1], prevBprice * 2**(1/nIter))
        if mode == 'exp': exp.host.withdraw(0)    
        if mode == 'exp': exp.host.deposit(0, depositTokens, [tok.balanceOf(exp.host) for tok in depositTokens])
        exp.farms.block += 60

    exp.farms.block += 360

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)

    users = [exp.host.name]
    balances = [exp.doku.balanceOf(exp.host) // 1e18]

    for user in exp.users:
        if exp.doku.balanceOf(user) != 0:
            users.append(user.name)
            balances.append(exp.doku.balanceOf(user) // 1e18)

    balName = 'balanceExp' if mode == 'exp' else 'balanceOk'
    df = pd.DataFrame({'user': users, balName: balances})
    df = df.set_index('user')
    df[balName] = df[balName].astype(int)

    return df, log_end
    

In [74]:
dfRes, logStart, logExp, logOk = doExp(exp, Exp1)
dfRes

,balanceExp,balanceOk
user,,
HOST,2720,2675
alice,2627,2675
bob,2606,2557
carol,3527,3527
david,6155,6202


### Price rise twice during 100 steps, HOST rebase after

In [43]:
def Exp2(exp, mode):
    assert mode in ('exp', 'ok')

    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    pid = None
    for i in range(len(exp.router.pools)):
        if exp.tokens[1] in exp.router.pools[i]['amounts']:
            pid = i
            break
    pool = exp.router.pools[pid]
    lp = list(pool['users'].items())[0][1]
    allLps = sum([i for _, i in list(pool['users'].items())])
    depositDict = {tok: amt * lp / allLps for tok, amt in pool['amounts'].items()}
    depositTokens, depositAmounts = list(depositDict.keys()), list(depositDict.values())

    exp.host.mintTokens(depositTokens, depositAmounts)

    exp.host.deposit(0, depositTokens, depositAmounts)

    exp.farms.block += 60
    
    nIter = 100
    for i in range(nIter):
        log_prev = exp.logState()
        prevBprice = exp.router.prices[exp.tokens[1]]
        exp.router.changeTokenPrice(exp.tokens[1], prevBprice * 2**(1/nIter))
        exp.farms.block += 60

    if mode == 'exp': exp.host.withdraw(0)    
    if mode == 'exp': exp.host.deposit(0, depositTokens, [tok.balanceOf(exp.host) for tok in depositTokens])

    exp.farms.block += 360

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)

    users = [exp.host.name]
    balances = [exp.doku.balanceOf(exp.host) // 1e18]

    for user in exp.users:
        if exp.doku.balanceOf(user) != 0:
            users.append(user.name)
            balances.append(exp.doku.balanceOf(user) // 1e18)

    balName = 'balanceExp' if mode == 'exp' else 'balanceOk'
    df = pd.DataFrame({'user': users, balName: balances})
    df = df.set_index('user')
    df[balName] = df[balName].astype(int)

    return df, log_end    

In [75]:
dfRes, logStart, logExp, logOk = doExp(exp, Exp2)
dfRes

,balanceExp,balanceOk
user,,
HOST,2680,2675
alice,2669,2675
bob,2562,2557
carol,3527,3527
david,6197,6202


### Price reduce twice during 100 steps, HOST rebase every time

In [45]:
def Exp3(exp, mode):
    assert mode in ('exp', 'ok')

    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    pid = None
    for i in range(len(exp.router.pools)):
        if exp.tokens[1] in exp.router.pools[i]['amounts']:
            pid = i
            break
    pool = exp.router.pools[pid]
    lp = list(pool['users'].items())[0][1]
    allLps = sum([i for _, i in list(pool['users'].items())])
    depositDict = {tok: amt * lp / allLps for tok, amt in pool['amounts'].items()}
    depositTokens, depositAmounts = list(depositDict.keys()), list(depositDict.values())

    exp.host.mintTokens(depositTokens, depositAmounts)

    exp.host.deposit(0, depositTokens, depositAmounts)

    exp.farms.block += 60
    
    nIter = 100
    for i in range(nIter):
        log_prev = exp.logState()
        prevBprice = exp.router.prices[exp.tokens[1]]
        exp.router.changeTokenPrice(exp.tokens[1], prevBprice * (1/2)**(1/nIter))
        if mode == 'exp': exp.host.withdraw(0)    
        if mode == 'exp': exp.host.deposit(0, depositTokens, [tok.balanceOf(exp.host) for tok in depositTokens])
        exp.farms.block += 60

    exp.farms.block += 360

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)

    users = [exp.host.name]
    balances = [exp.doku.balanceOf(exp.host) // 1e18]

    for user in exp.users:
        if exp.doku.balanceOf(user) != 0:
            users.append(user.name)
            balances.append(exp.doku.balanceOf(user) // 1e18)

    balName = 'balanceExp' if mode == 'exp' else 'balanceOk'
    df = pd.DataFrame({'user': users, balName: balances})
    df = df.set_index('user')
    df[balName] = df[balName].astype(int)

    return df, log_end
    

In [76]:
dfRes, logStart, logExp, logOk = doExp(exp, Exp3)
dfRes

,balanceExp,balanceOk
user,,
HOST,2663,2675
alice,2708,2675
bob,2501,2557
carol,3527,3527
david,6236,6202


### Price reduce twice during 100 steps, HOST rebase after

In [51]:
def Exp4(exp, mode):
    assert mode in ('exp', 'ok')

    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    pid = None
    for i in range(len(exp.router.pools)):
        if exp.tokens[1] in exp.router.pools[i]['amounts']:
            pid = i
            break
    pool = exp.router.pools[pid]
    lp = list(pool['users'].items())[0][1]
    allLps = sum([i for _, i in list(pool['users'].items())])
    depositDict = {tok: amt * lp / allLps for tok, amt in pool['amounts'].items()}
    depositTokens, depositAmounts = list(depositDict.keys()), list(depositDict.values())

    exp.host.mintTokens(depositTokens, depositAmounts)

    exp.host.deposit(0, depositTokens, depositAmounts)

    exp.farms.block += 60
    
    nIter = 100
    for i in range(nIter):
        log_prev = exp.logState()
        prevBprice = exp.router.prices[exp.tokens[1]]
        exp.router.changeTokenPrice(exp.tokens[1], prevBprice * (1/2)**(1/nIter))
        exp.farms.block += 60

    if mode == 'exp': exp.host.withdraw(0)    
    if mode == 'exp': exp.host.deposit(0, depositTokens, [tok.balanceOf(exp.host) for tok in depositTokens])

    exp.farms.block += 360

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)

    users = [exp.host.name]
    balances = [exp.doku.balanceOf(exp.host) // 1e18]

    for user in exp.users:
        if exp.doku.balanceOf(user) != 0:
            users.append(user.name)
            balances.append(exp.doku.balanceOf(user) // 1e18)

    balName = 'balanceExp' if mode == 'exp' else 'balanceOk'
    df = pd.DataFrame({'user': users, balName: balances})
    df = df.set_index('user')
    df[balName] = df[balName].astype(int)

    return df, log_end    
    

In [77]:
dfRes, logStart, logExp, logOk = doExp(exp, Exp4)
dfRes

,balanceExp,balanceOk
user,,
HOST,2674,2675
alice,2678,2675
bob,2551,2557
carol,3527,3527
david,6205,6202


# Mass Spawn

In [38]:
def massSpawn():    
    exp = Emulation()
    
    # spawn 20 tokens
    tokens_names = [f'T{i:02d}' for i in range(20)]

    # spawn 500 users
    users_names = [f'U{i:03d}' for i in range(500)]

    # token prices linear increase
    price_list = sorted([i*10 for i in range(1,11)] * 2, reverse=True)

    # lets say they are same as prices
    point_list = sorted([i*10 for i in range(1,11)] * 2, reverse=True)

    exp.reload(tokens_names, users_names, price_list, point_list)

    # random distribute tokens through users
    for user in exp.users:
        user_has_tokens = np.random.randint(0, 2, len(tokens_names)).tolist()
        amounts = np.random.lognormal(3, 1, len(tokens_names)).round().tolist()
        user.mintTokens(exp.tokens, amounts)

    # random spawn pools (each user either creates new or deposit to existing)
    for user in exp.users:
        do_create = np.random.choice([True, False])
        if do_create or len(exp.router.pools) < 1:
            tokens = np.array(list(user.assets.keys()))
            weights = np.random.randint(1, 11, len(tokens))
            amounts = np.array(list(user.assets.values()))

            valid = amounts != 0
            tokens, weights, amounts = tokens[valid], weights[valid], amounts[valid]

            user.createPool(tokens.tolist(), (weights / weights.sum()).tolist(), amounts.tolist())

        else:
            tokens = np.array(list(user.assets.keys()))
            amounts = np.array(list(user.assets.values()))

            valid = amounts != 0
            tokens, amounts = tokens[valid], amounts[valid]

            pid = np.random.choice(np.arange(len(exp.router.pools)))

            user.deposit(pid, tokens.tolist(), amounts.tolist())

    return exp
    

In [79]:
massExp = massSpawn()

In [82]:
dfRes, logStart, logExp, logOk = doExp(massExp, Exp0)
dfRes

,balanceExp,balanceOk
user,,
HOST,41,368
U000,854,852
U001,777,776
U002,779,779
U003,1019,1019
U004,683,683
U005,679,679
U006,967,966
U007,428,427


In [83]:
dfRes, logStart, logExp, logOk = doExp(massExp, Exp1)
dfRes

,balanceExp,balanceOk
user,,
HOST,0,65
U000,152,151
U001,138,138
U002,138,138
U003,181,181
U004,121,121
U005,121,120
U006,172,172
U007,76,76


In [84]:
dfRes, logStart, logExp, logOk = doExp(massExp, Exp2)
dfRes

,balanceExp,balanceOk
user,,
HOST,62,65
U000,151,151
U001,138,138
U002,138,138
U003,181,181
U004,121,121
U005,120,120
U006,172,172
U007,76,76


In [85]:
dfRes, logStart, logExp, logOk = doExp(massExp, Exp3)
dfRes

,balanceExp,balanceOk
user,,
HOST,0,65
U000,152,151
U001,138,138
U002,138,138
U003,181,181
U004,121,121
U005,121,120
U006,172,172
U007,76,76


In [86]:
dfRes, logStart, logExp, logOk = doExp(massExp, Exp4)
dfRes

,balanceExp,balanceOk
user,,
HOST,62,65
U000,151,151
U001,138,138
U002,138,138
U003,181,181
U004,121,121
U005,120,120
U006,172,172
U007,76,76
